# GridWorld 환경에서의 Q-Learning

Q-learning은 Chris Watkins가 1989년 [박사 논문](http://www.cs.rhul.ac.uk/~chrisw/thesis.html)을 위해 개발했을 때 초기 RL 혁신이었습니다. 이전에 가치 및 정책 반복에 사용했던 전환 및 보상 매트릭스를 모르거나 모델링하지 않고 MDP를 제어하기 위한 증분 동적 프로그래밍을 도입했습니다. 부분. 수렴 증명은 3년 후 [왓킨스와 다얀](http://www.gatsby.ucl.ac.uk/~dayan/papers/wd92.html)에 의해 이루어졌습니다.
Q-학습은 행동-가치 함수 q를 직접 최적화하여 q*를 근사화합니다. 학습은 정책을 벗어나 진행됩니다. 즉, 알고리즘은 가치 함수만으로 암시되는 정책을 기반으로 작업을 선택할 필요가 없습니다. 그러나 수렴하려면 모든 상태-작업 쌍이 훈련 프로세스 전반에 걸쳐 계속 업데이트되어야 합니다. 이를 보장하는 간단한 방법은 ε-탐욕 정책을 사용하는 것입니다.

Q-learning 알고리즘은 주어진 에피소드 수에 대해 무작위 초기화 후 상태-동작 값 함수를 지속적으로 개선합니다. 각 시간 단계에서 ε-탐욕 정책을 기반으로 작업을 선택하고 학습률 α를 사용하여 다음과 같이 가치 함수를 업데이트합니다.

$$Q(S_t, A_t)\leftarrow Q(S_t, A_t) + \alpha\left[R_{t_1}+\gamma \max_a Q(S_{t+1},,a) − Q(S_t, A_t) \오른쪽]$$

알고리즘은 전환 확률을 알고 있기 때문에 예상 값을 계산하지 않습니다. ε-탐욕 정책에 의해 생성된 보상과 다음 상태에 대한 가치 함수의 현재 추정치로부터 Q 함수를 학습합니다.
이 추정치를 개선하기 위해 추정값 함수를 사용하는 것을 부트스트랩핑이라고 합니다. Q-학습 알고리즘은 TD 학습 알고리즘의 일부입니다. TD 학습은 에피소드에 대한 최종 보상을 받을 때까지 기다리지 않습니다. 대신 최종 보상에 더 가까운 중간 상태의 값을 사용하여 추정치를 업데이트합니다. 이 경우 중간 상태는 한 시간 앞서 있습니다.

노트북은 동적 프로그램 [예](01_gridworld_dynamic_programming.ipynb)의 3 x 4 상태 그리드를 사용하여 Q-학습 에이전트를 구축하는 방법을 보여줍니다.

## 가져오기 및 설정

In [1]:
%matplotlib inline

from pathlib import Path
from time import process_time
import numpy as np
import pandas as pd
from mdptoolbox import mdp
from itertools import product

## 그리드월드 설정

먼저 동적 프로그램 [예](01_gridworld_dynamic_programming.ipynb)에서와 같이 작은 그리드 세계를 만듭니다.

### 상태, 행동, 보상

In [2]:
grid_size = (3, 4)
blocked_cell = (1, 1)
baseline_reward = -0.02
absorbing_cells = {(0, 3): 1, (1, 3): -1}

In [3]:
actions = ['L', 'U', 'R', 'D']
num_actions = len(actions)
probs = [.1, .8, .1, 0]

In [4]:
to_1d = lambda x: np.ravel_multi_index(x, grid_size)
to_2d = lambda x: np.unravel_index(x, grid_size)

In [5]:
num_states = np.product(grid_size)
cells = list(np.ndindex(grid_size))
states = list(range(len(cells)))

In [6]:
cell_state = dict(zip(cells, states))
state_cell= dict(zip(states, cells))

In [7]:
absorbing_states = {to_1d(s):r for s, r in absorbing_cells.items()}
blocked_state = to_1d(blocked_cell)

In [8]:
state_rewards = np.full(num_states, baseline_reward)
state_rewards[blocked_state] = 0
for state, reward in absorbing_states.items():
    state_rewards[state] = reward

In [9]:
action_outcomes = {}
for i, action in enumerate(actions):
    probs_ = dict(zip([actions[j % 4] for j in range(i, num_actions + i)], probs))
    action_outcomes[actions[(i + 1) % 4]] = probs_

In [10]:
action_outcomes

{'U': {'L': 0.1, 'U': 0.8, 'R': 0.1, 'D': 0},
 'R': {'U': 0.1, 'R': 0.8, 'D': 0.1, 'L': 0},
 'D': {'R': 0.1, 'D': 0.8, 'L': 0.1, 'U': 0},
 'L': {'D': 0.1, 'L': 0.8, 'U': 0.1, 'R': 0}}

### 전환 매트릭스

In [11]:
def get_new_cell(state, move):
    cell = to_2d(state)
    if actions[move] == 'U':
        return cell[0] - 1, cell[1]
    elif actions[move] == 'D':
        return cell[0] + 1, cell[1]
    elif actions[move] == 'R':
        return cell[0], cell[1] + 1
    elif actions[move] == 'L':
        return cell[0], cell[1] - 1

In [12]:
state_rewards

array([-0.02, -0.02, -0.02,  1.  , -0.02,  0.  , -0.02, -1.  , -0.02,
       -0.02, -0.02, -0.02])

In [13]:
def update_transitions_and_rewards(state, action, outcome):
    if state in absorbing_states.keys() or state == blocked_state:
        transitions[action, state, state] = 1
    else:
        new_cell = get_new_cell(state, outcome)
        p = action_outcomes[actions[action]][actions[outcome]]
        if new_cell not in cells or new_cell == blocked_cell:
            transitions[action, state, state] += p
            rewards[action, state, state] = baseline_reward
        else:
            new_state= to_1d(new_cell)
            transitions[action, state, new_state] = p
            rewards[action, state, new_state] = state_rewards[new_state]

In [14]:
rewards = np.zeros(shape=(num_actions, num_states, num_states))
transitions = np.zeros((num_actions, num_states, num_states))
actions_ = list(range(num_actions))
for action, outcome, state in product(actions_, actions_, states):
    update_transitions_and_rewards(state, action, outcome)

In [15]:
rewards.shape, transitions.shape

((4, 12, 12), (4, 12, 12))

## Q-러닝

2,500개의 에피소드에 대해 에이전트를 훈련하고 학습률 α= 0.1, ε-탐욕 정책에 ε=0.05를 사용합니다.

In [16]:
max_episodes = 2500
alpha = .1
epsilon = .05
gamma = .99

그런 다음 상태-동작 값 함수를 [상태 수 및 동작 수] 차원의 NumPy 배열로 무작위로 초기화합니다.

In [17]:
Q = np.random.rand(num_states, num_actions)
skip_states = list(absorbing_states.keys())+[blocked_state]
Q[skip_states] = 0

알고리즘은 임의의 위치에서 시작하여 종료될 때까지 ε-탐욕 정책에 따라 진행되는 2,500개의 에피소드를 생성하고 Q-학습 규칙에 따라 가치 함수를 업데이트합니다.

In [18]:
start = process_time()
for episode in range(max_episodes):
    state = np.random.choice([s for s in states if s not in skip_states])
    while not state in absorbing_states.keys():
        if np.random.rand() < epsilon:
            action = np.random.choice(num_actions)
        else:
            action = np.argmax(Q[state])
        next_state = np.random.choice(states, p=transitions[action, state])
        reward = rewards[action, state, next_state]
        Q[state, action] += alpha * (reward + gamma * np.max(Q[next_state])-Q[state, action])
        state = next_state
process_time() - start

0.67793093

에피소드는 0.6초가 걸리며 이전 섹션의 값 반복 예제 결과에 상당히 가까운 값 함수로 수렴됩니다.

In [19]:
pd.DataFrame(np.argmax(Q, 1).reshape(grid_size)).replace(dict(enumerate(actions)))

,0,1,2,3
0,R,R,R,L
1,U,L,L,L
2,U,L,L,D


In [20]:
pd.DataFrame(np.max(Q, 1).reshape(grid_size))

,0,1,2,3
0,0.877331,0.898778,0.945285,0.000000
1,0.845945,0.000000,0.706657,0.000000
2,0.807957,0.775816,0.745815,0.574385


## PyMDP 도구 상자

### Q학습

In [21]:
start = process_time()
ql = mdp.QLearning(transitions=transitions,
                   reward=rewards,
                   discount=gamma,
                   n_iter=int(1e6))

ql.run()
f'Time: {process_time()-start:.4f}'

'Time: 10.8511'

In [22]:
policy = np.asarray([actions[i] for i in ql.policy])
pd.DataFrame(policy.reshape(grid_size))

,0,1,2,3
0,R,R,R,L
1,U,L,U,L
2,R,R,U,D


In [23]:
value = np.asarray(ql.V).reshape(grid_size)
pd.DataFrame(value)

,0,1,2,3
0,0.752821,0.913639,0.963127,0.00000
1,0.464707,0.000000,0.710295,0.00000
2,0.227155,0.501195,0.603085,0.23838
